이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.5.0

**Spark Session**을 하나 만든다

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
        .getOrCreate()

In [ ]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2025-11-24 00:25:06--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.229.66, 52.218.183.98, 52.92.168.250, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.229.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv.3’

appl_stock.csv.3    100%[===================>] 139.78K  --.-KB/s    in 0.08s   

2025-11-24 00:25:06 (1.68 MB/s) - ‘appl_stock.csv.3’ saved [143130/143130]



In [ ]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [ ]:
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partitions before: 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [ ]:
df2 = df.repartition(4)
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [ ]:
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [ ]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [ ]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [ ]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [ ]:
!ls -tl dataOutput/

total 12
drwxr-xr-x 2 root root 4096 Nov 24 00:25 json
drwxr-xr-x 2 root root 4096 Nov 24 00:25 parquet
drwxr-xr-x 2 root root 4096 Nov 24 00:25 avro


In [ ]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 root root     0 Nov 24 00:25 _SUCCESS
-rw-r--r-- 1 root root 28024 Nov 24 00:25 part-00003-99e80232-cd0d-45d1-bc5c-9038bf4b4592-c000.snappy.parquet
-rw-r--r-- 1 root root 27833 Nov 24 00:25 part-00000-99e80232-cd0d-45d1-bc5c-9038bf4b4592-c000.snappy.parquet
-rw-r--r-- 1 root root 28069 Nov 24 00:25 part-00002-99e80232-cd0d-45d1-bc5c-9038bf4b4592-c000.snappy.parquet
-rw-r--r-- 1 root root 27787 Nov 24 00:25 part-00001-99e80232-cd0d-45d1-bc5c-9038bf4b4592-c000.snappy.parquet


In [ ]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 root root     0 Nov 24 00:25 _SUCCESS
-rw-r--r-- 1 root root 89029 Nov 24 00:25 part-00000-07176697-823e-418f-a280-fb8c37640ad8-c000.avro


In [ ]:
!ls -tl dataOutput/json/

total 240
-rw-r--r-- 1 root root      0 Nov 24 00:25 _SUCCESS
-rw-r--r-- 1 root root 122570 Nov 24 00:25 part-00000-d3e3d8d3-859e-4020-8adc-41c931d44d49-c000.json
-rw-r--r-- 1 root root 122814 Nov 24 00:25 part-00001-d3e3d8d3-859e-4020-8adc-41c931d44d49-c000.json
